In [ ]:
te = @__DIR__

sr = joinpath(dirname(te), "src")

_get_prefix(na) = splitext(na)[1]

display(
    symdiff(
        (_get_prefix(na) for na in readdir(sr) if endswith(na, r"\.jl$") && na != "BioLab.jl"),
        (
            _get_prefix(na) for
            na in readdir(te) if endswith(na, r"\.ipynb$") && na != "runtests.ipynb"
        ),
    ),
)

using BioLab

display(BioLab.TE)

display(BioLab.CA_)

display(BioLab.RA)

BioLab.IPYNB.run(@__DIR__, ("$pr.ipynb" for pr in ("runtests",)))